In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import ftplib
import os
import imageio.v3 as iio
import datetime
from glob import glob

In [2]:
# Define start and end dates
start = '2023-08-22'
end = '2023-08-24'

In [3]:
# load data
d = xr.open_dataset('/Users/colepietsch/Documents/EV333_AtmosphericDynamics/Rivers/atmorivers/ERA5_SouthAmerica_2023_hourly_tp.nc')

#subset data and convert to mm/hr
d = d.sel(time = slice(start, end))
d = d['tp'] * 1000 * 24

#defining number of frames given by subsetted data
num = len(d['time'])

In [5]:
#choosing data vis parameters and projection
cmap = cmocean.cm.rain
lev = np.arange(0, 10, 0.25)
proj = ccrs.PlateCarree(central_longitude=-77)

#create loop
for i in range(0, 1) :

    # define figure and axes, figure size, and resolution
    fig = plt.figure(figsize=(8, 4.5), dpi=300)
    ax = plt.axes(projection = proj)

    # filled contour map of metric in question
    d[i].plot.contourf(
        x = 'longitude',
        y = 'latitude',
        ax=ax,
        transform=ccrs.PlateCarree(),
        levels=lev,
        extend='max',
        colors=cmap,
        add_colorbar=True,
        cbar_kwargs = {"label":"Total Precip. in mm/hr"})

    # add coastlines
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.coastlines(resolution='110m')  #Currently can be one of “110m”, “50m”, and “10m”.

    # add grid lines
    gl = ax.gridlines(crs=ccrs.PlateCarree(),
                      draw_labels=True,
                      linewidth=1,
                      color='gray',
                      alpha=0.5,
                      linestyle='--')

    # Extract year, month, day, and hour from time series, splits are used, format 'YYYY-MM-DDT00:00:00.000000000'
    times = d['time'][i].values.astype('datetime64[s]')
    yr = str(np.datetime64(times, 'Y').astype(str))
    mon = str(np.datetime64(times, 'M').astype(str).split('-')[1])
    day = str(np.datetime64(times, 'D').astype(str).split('-')[2])
    hr = str(np.datetime64(times, 'h').astype(str).split('T')[1].split(':')[0])
   
    # Construct the filename using the extracted information
    name = f'frames/rivers_test_{yr}-{mon}-{day}-{hr}.png'

    # add title
    ax.set_title('Total Precipitation on ' + mon + '/' + day + ', ' + yr + ' at ' + hr + ':00')
  
    # Save the figure
    fig.savefig(name, facecolor='white', transparent=False, bbox_inches='tight')


    
    plt.close(fig)


In [ ]:
#getting frames and removing nonframes
files = sorted(glob('frames/rivers_test*.png'))

In [ ]:

#creating empty array
images = [ ]

#setting loop to append files to images, creating gif file
for filename in files :
  images.append(iio.imread(filename))

iio.imwrite('rivers_test.gif', images, duration = 250, loop = 0)
